In [1]:
import pandas as pd
import numpy as np
import math

import keras
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense
from keras.optimizers import Adam

import numpy as np
import random
from collections import deque

import sys

/Users/xiongzhuoran/anaconda3/envs/python3.4/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [6]:
# prints formatted price
def formatPrice(n):
	return ("-$" if n < 0 else "$") + "{0:.2f}".format(abs(n))

# returns the vector containing stock data from a fixed file
def getStockDataVec(key):
	vec = []
	lines = open("data/" + key + ".csv", "r").read().splitlines()

	for line in lines[1:]:
		vec.append(float(line.split(",")[4]))

	return vec

# returns the sigmoid
def sigmoid(x):
	return 1 / (1 + math.exp(-x))

# returns an an n-day state representation ending at time t
def getState(data, t, n):
	d = t - n + 1
	block = data[d:t + 1] if d >= 0 else -d * [data[0]] + data[0:t + 1] # pad with t0
	res = []
	for i in range(n - 1):
		res.append(sigmoid(block[i + 1] - block[i]))

	return np.array([res])

In [7]:
stock_name, episode_count = ['^GSPC', 1000]

agent = Agent()
data = getStockDataVec(stock_name)
l = len(data) - 1
batch_size = 32

for e in range(episode_count + 1):
	print("Episode " + str(e) + "/" + str(episode_count))
	state = getState(data, 0)

	total_profit = 0
	agent.inventory = []

	for t in range(l):
		action = agent.act(state)

		# sit
		next_state = getState(data, t + 1, window_size + 1)
		reward = 0

		if action == 1: # buy
			agent.inventory.append(data[t])
			print("Buy: " + formatPrice(data[t]))

		elif action == 2 and len(agent.inventory) > 0: # sell
			bought_price = agent.inventory.pop(0)
			reward = max(data[t] - bought_price, 0)
			total_profit += data[t] - bought_price
			print("Sell: " + formatPrice(data[t]) + " | Profit: " + formatPrice(data[t] - bought_price))

		done = True if t == l - 1 else False
		agent.memory.append((state, action, reward, next_state, done))
		state = next_state

		if done:
			print("--------------------------------")
			print("Total Profit: " + formatPrice(total_profit))
			print("--------------------------------")

		if len(agent.memory) > batch_size:
			agent.expReplay(batch_size)

	if e % 10 == 0:
		agent.model.save("models/model_ep" + str(e))

Episode 0/1000
Buy: $1333.34
Sell: $1298.35 | Profit: -$34.99
Buy: $1295.86
Sell: $1313.27 | Profit: $17.41
Buy: $1326.82
Buy: $1318.55
Buy: $1326.65
Buy: $1329.47
Sell: $1342.54 | Profit: $15.72
Buy: $1342.90
Sell: $1360.40 | Profit: $41.85
Buy: $1364.30
Buy: $1354.95
Buy: $1364.17
Buy: $1373.73
Buy: $1366.01
Sell: $1373.47 | Profit: $46.82
Sell: $1349.47 | Profit: $20.00
Sell: $1352.26 | Profit: $9.36
Sell: $1340.89 | Profit: -$23.41
Sell: $1314.76 | Profit: -$40.19
Sell: $1330.31 | Profit: -$33.86
Buy: $1326.61
Buy: $1278.94
Buy: $1255.27
Buy: $1252.82
Sell: $1245.86 | Profit: -$127.87
Buy: $1267.65
Sell: $1257.94 | Profit: -$108.07
Sell: $1234.18 | Profit: -$92.43
Buy: $1241.41
Sell: $1264.74 | Profit: -$14.20
Buy: $1233.42
Buy: $1197.66
Buy: $1166.71
Buy: $1173.56
Sell: $1150.53 | Profit: -$104.74
Sell: $1142.62 | Profit: -$110.20
Buy: $1122.14
Sell: $1117.58 | Profit: -$150.07
Buy: $1139.83
Buy: $1152.69
Sell: $1182.17 | Profit: -$59.24
Sell: $1153.29 | Profit: -$80.13
Buy: $1160

Sell: $773.14 | Profit: -$114.54
Sell: $764.90 | Profit: -$131.34
Sell: $752.83 | Profit: -$63.38
Sell: $735.09 | Profit: -$113.72
Sell: $700.82 | Profit: -$169.92
Sell: $696.33 | Profit: -$148.89
Sell: $712.87 | Profit: -$163.20
Sell: $682.55 | Profit: -$227.15
Sell: $683.38 | Profit: -$205.29
Sell: $676.53 | Profit: -$222.71
Sell: $719.60 | Profit: -$153.99
Sell: $721.36 | Profit: -$158.37
Sell: $750.74 | Profit: -$117.83
Sell: $756.55 | Profit: -$156.63
Sell: $753.89 | Profit: -$150.53
Sell: $778.12 | Profit: -$107.16
Sell: $794.35 | Profit: -$93.53
Sell: $784.04 | Profit: -$87.59
Sell: $768.54 | Profit: -$94.62
Sell: $822.92 | Profit: -$45.23
Sell: $806.12 | Profit: -$66.68
Sell: $813.88 | Profit: -$55.54
Sell: $832.86 | Profit: -$57.78
Sell: $815.94 | Profit: -$87.31
Buy: $787.53
Sell: $797.87 | Profit: -$133.93
Sell: $811.08 | Profit: -$116.37
Sell: $834.38 | Profit: -$100.32
Sell: $842.50 | Profit: -$64.15
Sell: $835.48 | Profit: -$74.25
Sell: $815.55 | Profit: -$74.80
Sell: $82

Buy: $1003.45
Sell: $1012.27 | Profit: $8.82
Buy: $1133.06
Buy: $1100.64
Sell: $1113.57 | Profit: -$19.49
Sell: $1130.20 | Profit: $29.56
Buy: $797.70
Buy: $843.43
Sell: $838.68 | Profit: $40.98
Sell: $852.84 | Profit: $9.41
Buy: $882.15
Sell: $890.71 | Profit: $8.56
Buy: $821.99
Sell: $829.85 | Profit: $7.86
Buy: $984.84
Buy: $997.48
Sell: $998.51 | Profit: $13.67
Sell: $988.61 | Profit: -$8.87
Buy: $1033.78
Buy: $1038.73
Sell: $1038.06 | Profit: $4.28
Sell: $1045.35 | Profit: $6.62
Buy: $1147.62
Buy: $1143.94
Sell: $1141.55 | Profit: -$6.07
Sell: $1155.37 | Profit: $11.43
Buy: $1095.70
Buy: $1084.10
Sell: $1091.49 | Profit: -$4.21
Sell: $1088.68 | Profit: $4.58
Buy: $1065.22
Buy: $1079.04
Sell: $1075.79 | Profit: $10.57
Sell: $1063.23 | Profit: -$15.81
Buy: $1181.76
Buy: $1182.65
Buy: $1178.57
Sell: $1190.33 | Profit: $8.57
Sell: $1191.17 | Profit: $8.52
Sell: $1190.25 | Profit: $11.68
Buy: $1200.08
Sell: $1206.83 | Profit: $6.75
Buy: $1210.96
Buy: $1216.96
Sell: $1216.10 | Profit: $

Sell: $1494.25 | Profit: $13.84
Buy: $1539.59
Buy: $1542.84
Sell: $1557.59 | Profit: $18.00
Sell: $1552.58 | Profit: $9.74
Buy: $1273.37
Buy: $1320.65
Sell: $1308.77 | Profit: $35.40
Sell: $1315.48 | Profit: -$5.17
Buy: $1267.79
Buy: $1224.51
Sell: $1232.04 | Profit: -$35.75
Sell: $1249.05 | Profit: $24.54
Buy: $788.42
Buy: $778.94
Sell: $770.05 | Profit: -$18.37
Sell: $743.33 | Profit: -$35.61
Buy: $753.89
Sell: $784.04 | Profit: $30.15
Buy: $940.09
Sell: $939.14 | Profit: -$0.95
Buy: $1064.66
Buy: $1071.66
Sell: $1060.87 | Profit: -$3.79
Sell: $1050.78 | Profit: -$20.88
Buy: $1091.49
Buy: $1095.63
Sell: $1108.86 | Profit: $17.37
Sell: $1109.24 | Profit: $13.61
Buy: $1135.68
Buy: $1136.94
Sell: $1120.80 | Profit: -$14.88
Sell: $1115.05 | Profit: -$21.89
Buy: $1093.67
Buy: $1102.66
Sell: $1115.01 | Profit: $21.34
Sell: $1113.84 | Profit: $11.18
--------------------------------
Total Profit: $83.74
--------------------------------
Episode 6/1000
Buy: $1298.35
Buy: $1295.86
Sell: $1300.8

Sell: $1050.07 | Profit: $0.59
Buy: $1095.89
Buy: $1109.48
Sell: $1109.64 | Profit: $13.75
Sell: $1111.92 | Profit: $2.44
Buy: $1117.49
Buy: $1119.55
Sell: $1121.53 | Profit: $4.04
Sell: $1113.99 | Profit: -$5.56
Buy: $1086.20
Buy: $1084.07
Sell: $1094.83 | Profit: $8.63
Sell: $1095.42 | Profit: $11.35
Buy: $1190.33
Buy: $1191.17
Sell: $1190.25 | Profit: -$0.08
Sell: $1177.07 | Profit: -$14.10
Buy: $1201.72
Buy: $1202.30
Buy: $1191.99
Sell: $1197.01 | Profit: -$4.71
Sell: $1205.30 | Profit: $3.00
Sell: $1206.14 | Profit: $14.15
Buy: $1181.39
Sell: $1184.07 | Profit: $2.68
Buy: $1194.94
Buy: $1197.87
Sell: $1211.86 | Profit: $16.92
Sell: $1219.44 | Profit: $21.57
Buy: $1254.42
Buy: $1248.29
Sell: $1273.46 | Profit: $19.04
Sell: $1273.48 | Profit: $25.19
Buy: $1242.28
Buy: $1236.20
Sell: $1234.49 | Profit: -$7.79
Sell: $1236.86 | Profit: $0.66
Buy: $1325.18
Buy: $1318.03
Sell: $1314.78 | Profit: -$10.40
Sell: $1326.37 | Profit: $8.34
Buy: $1418.30
Buy: $1416.60
Sell: $1418.34 | Profit: $

Buy: $1408.66
Buy: $1423.57
Sell: $1425.35 | Profit: $16.69
Sell: $1426.63 | Profit: $3.06
Buy: $1274.98
Sell: $1236.83 | Profit: -$38.15
Buy: $911.29
Buy: $873.29
Sell: $850.75 | Profit: -$60.54
Sell: $859.12 | Profit: -$14.17
Buy: $843.74
Buy: $850.12
Sell: $805.22 | Profit: -$38.52
Sell: $840.24 | Profit: -$9.88
Buy: $721.36
Buy: $750.74
Sell: $756.55 | Profit: $35.19
Sell: $753.89 | Profit: $3.15
Buy: $979.62
Sell: $975.15 | Profit: -$4.47
Buy: $1092.91
Sell: $1079.60 | Profit: -$13.31
Buy: $1127.78
Buy: $1126.20
Sell: $1126.42 | Profit: -$1.36
Sell: $1115.10 | Profit: -$11.10
Buy: $1091.76
Sell: $1096.78 | Profit: $5.02
Buy: $1211.67
Buy: $1192.13
Sell: $1197.52 | Profit: -$14.15
Sell: $1207.17 | Profit: $15.04
Buy: $1064.59
Sell: $1048.92 | Profit: -$15.67
Buy: $1193.57
Buy: $1197.96
Sell: $1221.06 | Profit: $27.49
Sell: $1225.85 | Profit: $27.89
Buy: $1187.76
Sell: $1180.55 | Profit: -$7.21
--------------------------------
Total Profit: -$279.41
--------------------------------


KeyboardInterrupt: 

In [20]:
stock_name, model_name = ['^GSPC_2011','model_ep10']
model = load_model("models/" + model_name)
window_size = model.layers[0].input.shape.as_list()[1]

agent = Agent(window_size, True, model_name)
data = getStockDataVec(stock_name)
l = len(data) - 1
batch_size = 32

state = getState(data, 0, window_size + 1)
total_profit = 0
agent.inventory = []

In [21]:
for t in range(l):
#     print(t)
    action = agent.act(state)
    
#     print(action)
    
    # sit
    next_state = getState(data, t + 1, window_size + 1)
    reward = 0

    if action == 1: # buy
        agent.inventory.append(data[t])
        print("Buy: " + formatPrice(data[t]))

    elif action == 2 and len(agent.inventory) > 0: # sell
        bought_price = agent.inventory.pop(0)
        reward = max(data[t] - bought_price, 0)
        total_profit += data[t] - bought_price
        print("Sell: " + formatPrice(data[t]) + " | Profit: " + formatPrice(data[t] - bought_price))

    done = True if t == l - 1 else False
    agent.memory.append((state, action, reward, next_state, done))
    state = next_state

    if done:
        print("--------------------------------")
        print(stock_name + " Total Profit: " + formatPrice(total_profit))
        print("--------------------------------")


2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
--------------------------------
^GSPC_2011 Total Profit: $0.00
--------------------------------


In [19]:
state

array([[9.80353885e-01, 6.09846464e-07, 1.00000000e+00, 9.18333895e-01,
        9.99965690e-01, 9.99987992e-01, 5.25003626e-01, 1.38826742e-07,
        9.99998454e-01, 4.40744033e-03]])

In [9]:
load_model("models/" + model_name)